# Data Cleaning and extraction (LAMIA)


In [9]:
import pandas as pd
import json

In [6]:
!ls ../

README.md        figures          notebooks
datasets         ift6758          requirements.txt
environment.yml  ift6758.egg-info setup.py


In [7]:

df = pd.read_json('../datasets/raw/2016/playoffs/2016020101.json')

In [8]:
df

,copyright,gamePk,link,metaData,gameData,liveData
wait,NHL and the NHL Shield are registered trademar...,2016020101,/api/v1/game/2016020101/feed/live,1.000000e+01,NaN,NaN
timeStamp,NHL and the NHL Shield are registered trademar...,2016020101,/api/v1/game/2016020101/feed/live,2.017092e+13,NaN,NaN
game,NHL and the NHL Shield are registered trademar...,2016020101,/api/v1/game/2016020101/feed/live,NaN,"{'pk': 2016020101, 'season': '20162017', 'type...",NaN
datetime,NHL and the NHL Shield are registered trademar...,2016020101,/api/v1/game/2016020101/feed/live,NaN,"{'dateTime': '2016-10-27T23:30:00Z', 'endDateT...",NaN
status,NHL and the NHL Shield are registered trademar...,2016020101,/api/v1/game/2016020101/feed/live,NaN,"{'abstractGameState': 'Final', 'codedGameState...",NaN
teams,NHL and the NHL Shield are registered trademar...,2016020101,/api/v1/game/2016020101/feed/live,NaN,"{'away': {'id': 13, 'name': 'Florida Panthers'...",NaN
players,NHL and the NHL Shield are registered trademar...,2016020101,/api/v1/game/2016020101/feed/live,NaN,"{'ID8474709': {'id': 8474709, 'fullName': 'Mat...",NaN
venue,NHL and the NHL Shield are registered trademar...,2016020101,/api/v1/game/2016020101/feed/live,NaN,"{'id': 5015, 'name': 'Air Canada Centre', 'lin...",NaN
plays,NHL and the NHL Shield are registered trademar...,2016020101,/api/v1/game/2016020101/feed/live,NaN,NaN,{'allPlays': [{'result': {'event': 'Game Sched...
linescore,NHL and the NHL Shield are registered trademar...,2016020101,/api/v1/game/2016020101/feed/live,NaN,NaN,"{'currentPeriod': 3, 'currentPeriodOrdinal': '..."


In [10]:
# load data using Python JSON module
with open('../datasets/raw/2016/playoffs/2016020101.json','r') as f:
    data = json.loads(f.read())

In [40]:
#df_nested_list = pd.json_normalize(data, record_path =['gameData'])
def get_generalData(data):
    id_game = data['gameData']['game']['pk']
    season = data['gameData']['game']['season']
    dateTime =  data['gameData']['datetime']['dateTime']
    endDateTime = data['gameData']['datetime']['endDateTime']
    abstractGameState = data['gameData']['status']['abstractGameState']
    team_away_name = data['gameData']['teams']['away']['name']
    team_home_name = data['gameData']['teams']['home']['name']
    return [id_game,season,dateTime,endDateTime,abstractGameState,team_away_name,team_home_name]

In [45]:
match_data = get_generalData(data)
match_events_list = []
for item in data['liveData']['plays']['allPlays']:
    if item['result']['event'] not in ["Goal", "Shot"]:
        continue
    else :
        # event Idx
        event_Idx = item['about']['eventIdx']

        # periodTime
        periodTime = item['about']['periodTime']
        
        # team information (which team shot)
        id_team_that_shot = item['team']['id'] 
        name_team_that_shot = item['team']['name'] 

        # indicator if its a shot or a goal
        result_event = item['result']['event']
        
        # the on ice coordinates
        x_coord = item['coordinates']['x'] 
        y_coord = item['coordinates']['y']

        # the shooter and goalie name
        for item_bis in item['players']:
            if item_bis['playerType']=="Goalie":
                goalie_name = item_bis['player']["fullName"]
            elif item_bis['playerType'] in ["Shooter", "Scorer"]:
                shooter_name = item_bis['player']["fullName"]
            else:
                continue
        
        # shot type
        shot_type = item['result']['secondaryType']

        try : 
            # empty net
            empty_net = item['result']['emptyNet']
        except :
            empty_net = None

        
        # strength
        try :
            strength = item['result']['name']
        except :
            strength = None
        
        all_data = [event_Idx, periodTime, id_team_that_shot,name_team_that_shot,result_event,x_coord,y_coord,goalie_name,shooter_name,shot_type,empty_net,strength]
        match_events_list.append(all_data+match_data)


 

In [48]:
columns_name = [
"event_Idx", "periodTime",\
     "id_team_that_shot","name_team_that_shot","result_event","x_coord","y_coord","goalie_name","shooter_name","shot_type","empty_net",\
        "strength","id_game","season","dateTime","endDateTime","abstractGameState","team_away_name","team_home_name"]

In [49]:

pd.DataFrame(match_events_list, columns=columns_name)

,event_Idx,periodTime,id_team_that_shot,name_team_that_shot,result_event,x_coord,y_coord,goalie_name,shooter_name,shot_type,empty_net,strength,id_game,season,dateTime,endDateTime,abstractGameState,team_away_name,team_home_name
0,6,00:43,10,Toronto Maple Leafs,Shot,-76.0,14.0,Roberto Luongo,James van Riemsdyk,Backhand,None,None,2016020101,20162017,2016-10-27T23:30:00Z,2016-10-28T02:04:25Z,Final,Florida Panthers,Toronto Maple Leafs
1,20,03:09,13,Florida Panthers,Shot,62.0,-12.0,Frederik Andersen,Jason Demers,Snap Shot,None,None,2016020101,20162017,2016-10-27T23:30:00Z,2016-10-28T02:04:25Z,Final,Florida Panthers,Toronto Maple Leafs
2,25,03:33,13,Florida Panthers,Shot,32.0,21.0,Frederik Andersen,Michael Matheson,Slap Shot,None,None,2016020101,20162017,2016-10-27T23:30:00Z,2016-10-28T02:04:25Z,Final,Florida Panthers,Toronto Maple Leafs
3,27,03:49,13,Florida Panthers,Shot,35.0,29.0,Frederik Andersen,Aleksander Barkov,Snap Shot,None,None,2016020101,20162017,2016-10-27T23:30:00Z,2016-10-28T02:04:25Z,Final,Florida Panthers,Toronto Maple Leafs
4,28,03:56,13,Florida Panthers,Shot,30.0,28.0,Frederik Andersen,Michael Matheson,Slap Shot,None,None,2016020101,20162017,2016-10-27T23:30:00Z,2016-10-28T02:04:25Z,Final,Florida Panthers,Toronto Maple Leafs
5,30,04:06,10,Toronto Maple Leafs,Shot,-26.0,-14.0,Roberto Luongo,Morgan Rielly,Snap Shot,None,None,2016020101,20162017,2016-10-27T23:30:00Z,2016-10-28T02:04:25Z,Final,Florida Panthers,Toronto Maple Leafs
6,42,05:44,13,Florida Panthers,Shot,6.0,2.0,Frederik Andersen,Aaron Ekblad,Snap Shot,None,None,2016020101,20162017,2016-10-27T23:30:00Z,2016-10-28T02:04:25Z,Final,Florida Panthers,Toronto Maple Leafs
7,47,05:57,10,Toronto Maple Leafs,Shot,-68.0,-27.0,Roberto Luongo,Zach Hyman,Backhand,None,None,2016020101,20162017,2016-10-27T23:30:00Z,2016-10-28T02:04:25Z,Final,Florida Panthers,Toronto Maple Leafs
8,56,06:55,13,Florida Panthers,Goal,58.0,24.0,Frederik Andersen,Jonathan Marchessault,Slap Shot,False,None,2016020101,20162017,2016-10-27T23:30:00Z,2016-10-28T02:04:25Z,Final,Florida Panthers,Toronto Maple Leafs
9,58,07:31,13,Florida Panthers,Shot,66.0,-4.0,Frederik Andersen,Jonathan Marchessault,Tip-In,None,None,2016020101,20162017,2016-10-27T23:30:00Z,2016-10-28T02:04:25Z,Final,Florida Panthers,Toronto Maple Leafs
